Quarto -- a versatile tool from R world! I have used a lot of its precursor, R Markdown back in the days -- even if I did not use R daily as a graduate student. Quarto is pretty similar to it with more functionalities. I make quizzes and exams with Quarto on RStudio. It seems like RStudio (now renamed Posit) guys' goal now is expanding to Python world.

Project -- you have access to previous topics in PIC16B! [link](https://docs.google.com/spreadsheets/d/1oEajTpOU6YYm5NdVAeX33IZOAwgBU0ZmvAkDGFTCY7U/edit?usp=sharing)

So far: merging, melting, pivoting;
after groupby-- aggregation, transform, apply


# Working with Databases

### Takeaways for Today

1. Reading large data sets into memory can be surprisingly slow, and in some cases completely impractical. 
2. Databases offer a convenient way to perform selective *queries* on our data, allowing us to retrieve only the data that we want at any given time. 
3. The `sqlite3` package enables us to work with databases using Python commands, and the `pandas` package makes it easy to read the results of database queries as data frames. 

## Intro 

In class, we will mostly work with the data that could easily fit in memory. Data sets of under 20 megabytes or so can be loaded easily on most modern laptops, although subsequent analysis activities may be slow. 

Larger data sets can be noticeably slow to even load into memory. So far, we worked with NOAA-GHCN data. When loaded in memory, it took up ~1 GB. This is not too big to fit in memory, but it's large enough that it's noticeably slow to even load into pandas. To accelerate the process, let's download the files to the computer now. 


In [1]:
import os
# create folder named "datafiles" if it does not exist
if not os.path.exists("datafiles"): 
    os.mkdir("datafiles")

# download the files
import urllib.request
intervals = [f"{10 * i + 1}-{10 * (i+1)}" for i in range(190, 202)]
for interval in intervals:
    url = f"https://raw.githubusercontent.com/PIC16B-ucla/24F/main/datasets/noaa-ghcn/decades/{interval}.csv"
    urllib.request.urlretrieve(url, f"datafiles/{interval}.csv")

Import pandas, and measure time for `read_csv()` part only:

In [2]:
import pandas as pd

In [5]:
%%time

intervals = [f"{10 * i + 1}-{10 * (i+1)}" for i in range(190, 202)]# quiz! 1901-1910 to 2011-2020.
dfs = []
for interval in intervals:
    filepath = f"datafiles/{interval}.csv"
    df = pd.read_csv(filepath)
    dfs.append(df)
df = pd.concat(dfs, axis=0, ignore_index=True)

CPU times: user 621 ms, sys: 149 ms, total: 769 ms
Wall time: 805 ms


Longer than half a second just to load the data set! 

Note: Today, I'm not even *down*loading the data set -- this is just how long it takes to move the data from my computer's SSD into RAM. There's no `prepare_df()`, either.

We can get a pretty good estimate of how much RAM is required to store the data using a handy method of Pandas data frames: 


In [6]:
df.shape

(1350617, 14)

In [7]:
df.head()

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,AG000060390,1901,1034.0,844.0,1164.0,1534.0,1564.0,2354.0,2444.0,2364.0,2214.0,1604.0,1364.0,1004.0
1,AG000060390,1902,984.0,1194.0,1254.0,1544.0,1514.0,1984.0,2484.0,2574.0,2164.0,1694.0,1494.0,1134.0
2,AG000060390,1903,1144.0,1064.0,1266.0,1346.0,1646.0,1906.0,2326.0,2446.0,2146.0,1926.0,1356.0,1056.0
3,AG000060390,1904,926.0,1126.0,1136.0,1406.0,1836.0,2126.0,2516.0,2606.0,2076.0,1826.0,1376.0,1246.0
4,AG000060390,1905,906.0,866.0,1386.0,1576.0,1596.0,2146.0,2326.0,2496.0,2206.0,1716.0,1386.0,1076.0


In [8]:
df

,ID,Year,VALUE1,VALUE2,VALUE3,VALUE4,VALUE5,VALUE6,VALUE7,VALUE8,VALUE9,VALUE10,VALUE11,VALUE12
0,AG000060390,1901,1034.0,844.0,1164.0,1534.0,1564.0,2354.0,2444.0,2364.0,2214.0,1604.0,1364.0,1004.0
1,AG000060390,1902,984.0,1194.0,1254.0,1544.0,1514.0,1984.0,2484.0,2574.0,2164.0,1694.0,1494.0,1134.0
2,AG000060390,1903,1144.0,1064.0,1266.0,1346.0,1646.0,1906.0,2326.0,2446.0,2146.0,1926.0,1356.0,1056.0
3,AG000060390,1904,926.0,1126.0,1136.0,1406.0,1836.0,2126.0,2516.0,2606.0,2076.0,1826.0,1376.0,1246.0
4,AG000060390,1905,906.0,866.0,1386.0,1576.0,1596.0,2146.0,2326.0,2496.0,2206.0,1716.0,1386.0,1076.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350612,ZI000067983,2012,2186.0,NaN,2157.0,1833.0,1835.0,1625.0,1563.0,NaN,2034.0,NaN,2298.0,2199.0
1350613,ZI000067983,2013,NaN,2163.0,2054.0,1870.0,1729.0,1686.0,1528.0,1862.0,2100.0,1967.0,2258.0,2142.0
1350614,ZI000067983,2014,NaN,2187.0,NaN,NaN,NaN,NaN,NaN,NaN,2087.0,2110.0,2297.0,NaN
1350615,ZI000067983,2015,2175.0,2251.0,2083.0,1952.0,1880.0,1670.0,1650.0,1810.0,2010.0,2290.0,2230.0,2405.0


`df.memory_usage()` shows memory usage of each column in bytes. 

In [13]:
df.memory_usage(index=True).sum() / 10**6

151.269236

**POLL**: Which of the following is closest to this number?

- A. 10 MB
- B. 100 MB
- C. 1 GB
- D. 10 GB

Compare this number to the size of the long format table we used Wednesday.



The data contains 1.35M rows and 14 columns. Altogether, these consume roughly ____MB of RAM. This is an appreciable fraction of the RAM on most laptops. In some cases, we might need to work with even larger data sets that don't fit in RAM at all. 

## Introducing Databases

When dealing with large data sets like these, it's relatively rare that we absolutely *have* to operate on the entire data set. In most cases, we can work with parts of the data at a time. In the context of the temperature data, 

1. We might want temperature measurements only between years 1990-2020. 
2. We might want temperature measurements only for a certain set of countries - maybe in Asia, say. 
3. We might only want temperature measurements only in the month of March. 
4. We might want a random 1% of all the data. 

**Databases** provide us with a structured way to move subsets of data from storage into memory. Python has a built-in module called `sqlite3` which we can use to create, manipulate, and query databases. There's also a very handy `pandas` interface, enabling us to efficiently create `pandas` data frames containing exactly the data that we want. 

- Database vs. memory (RAM): large dataset doesn't fit in RAM, large database on persistant storage doesn't disappear when you restart your computer.

- Database vs. text-based data file: database softwares are made for easy access/manipulation of data ("querying" the data). for example, no need to load csv file with 10 million rows just to look up one customer's information.

A common approach in genomics: compress data for each genotypes in a large data file, and store location of each genotype in that file in a separate database

### Creating and Populating Databases

After importing the `sqlite3` module, the first thing we should do is connect to a database. In case the specified database does not exist, instantiating the connection will also create an empty database with the specified name.  

In [14]:
import sqlite3

In [15]:
conn = sqlite3.connect("temps.db") # create a database in current directory called temps.db

A quick check in our file directory reveals that we now have a file called `temps.db`. Great! 

There are many ways to add data to a database. Since we are already familiar with Pandas data frames, we'll make use of some extremely convenient functionality which allows us to directly write data frames to a database. 

__But wait!__ Wasn't the whole point of this to avoid reading in an entire data frame? Indeed! 

Let's process each csv file one at a time, using the `prepare_df` function we used in the previous lecture.

In [16]:
def prepare_df(df):
    """
    prepares a piece of wide format dataframe into a long format data frame
    """
    # melt to the long format table
    df = df.melt(
        id_vars = ["ID", "Year"],
        value_vars = [f"VALUE{i}" for i in range(1, 13)],
        var_name = "Month",
        value_name = "Temp"
    )

    # cleaning month and temp
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100

    return df

OK! We are finally ready to populate a *table* in our database. 

A relational database consists of several *table*s. They are data frame-like objects, represented in SQLite (a type of RDBMS, relational database management system) rather than in Python. The `df.to_sql()` method writes to a specified table in the database (the `conn` object from earlier). We need to specify `if_exists` to ensure that we add each piece to the table, rather than overwriting them each time. 

In [17]:
intervals = [f"{10 * i + 1}-{10 * (i+1)}" for i in range(190, 202)]
for i, interval in enumerate(intervals):
    filepath = f"datafiles/{interval}.csv"
    df = pd.read_csv(filepath)
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "replace" if i == 0 else "append", index = False)

Let's similarly add a table for the metadata in our database. This is a pretty small data set so we don't need to worry about reading it in by chunks.

In [18]:
filename = "https://raw.githubusercontent.com/PIC16B-ucla/24F/refs/heads/main/datasets/noaa-ghcn/station-metadata.csv"
stations = pd.read_csv(filename)
stations.to_sql("stations", conn, if_exists = "replace", index=False)

27585

Now we have a database containing two tables. Let's just check that this is indeed the case. 
<!--
SELECT name FROM sqlite_master WHERE type='table'
-->

In [19]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('temperatures',), ('stations',)]


Ok wait -- there's actually a lot that just happened here! 

First: the *cursor* is our primary way to interact with the database. The cursor `execute`s *SQL commands*. Structured Query Language (SQL) is actually its own mini-programming language specifically designed for interacting with databases. We do need to learn a bit of SQL in order to work with databases.

Here are some resources:

- https://www.w3schools.com/sql/

- https://www.sqltutorial.org/

- https://www.w3schools.com/sql/sql_quickref.asp

- https://www.sqltutorial.org/sql-cheat-sheet/

or you can just get the hang of the basics and google the rest as you need, which is what 99% of people do.



Now let's take a look at the actual command that we `execute`d.

```sql
SELECT name FROM sqlite_master WHERE type='table'
```

Let's break this down: 

- `SELECT name`: Show me the entries in the `name` column
- `FROM sqlite_master`: of the `sqlite_master` table
- `WHERE type='table'`: subject to the condition that the entry in the `type` column of `sqlite_master` is equal to ``table`` (i.e. don't include other kinds of objects)

Finally, `cursor.fetchall()` returns the a list containing all the items returned by the query, which we then print.

The special `sqlite_master` table contains information on all the objects in the database. We don't usually query it when we want to obtain data, but we can query it to learn about what kinds of tables we have, what their columns are, etc. 

Let's get more detailed information about the items in each table. For example, we can inspect the column names and data types in each. This is a good way to check that we actually populated our database correctly. 

Let select the column `sql` column from each table names in the master table


In [20]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)


This looks pretty good! We have two tables, called `temperatures` and `stations`. The column names are what we would expect. Notice that `sql` has automatically inferred the data types, such as `REAL` and `TEXT`, from the input. Handy! 

#### Basic Queries

Now we're ready to perform some basic queries on the data tables themselves. 

What data do we have for the year 1990? In `pandas` we would do this like: 

```python
df[df["Year"] == 1990]
```

The SQL syntax is very different, but contains all the same ideas. 

- `SELECT`, like the syntax `temperatures["id"]`, controls which column(s) will be returned. 
- `FROM` tells us which table to return columns from. 
- `WHERE` is like the Boolean index `[temperatures["year"] == 1990]`. Only rows in which this criterion is satisfied will be returned. 

**NOTE**: SQL commands are case-insensitive, but it's considered good practice to place SQL keywords in ALL CAPS while names of columns and other content goes in lowercase. 

Let's quickly go over some of the basic SQL queries using the following data.

In [21]:
employees = pd.DataFrame(data={"name"   : ["Alice", "Bob", "Carol", "Dave", "Eve", "Frank"],
                    "email"  : ["alice@company.com", "bob@company.com",
                               "carol@company.com", "dave@company.com",
                               "eve@company.com",   "frank@comany.com"],
                    "salary" : [52000, 40000, 30000, 33000, 44000, 37000],
                    "dept"   : ["Accounting", "Accounting", "Sales",
                               "Accounting", "Sales", "Sales"]
                              }
                        )    

In [22]:
employees

,name,email,salary,dept
0,Alice,alice@company.com,52000,Accounting
1,Bob,bob@company.com,40000,Accounting
2,Carol,carol@company.com,30000,Sales
3,Dave,dave@company.com,33000,Accounting
4,Eve,eve@company.com,44000,Sales
5,Frank,frank@comany.com,37000,Sales


In [23]:
phone = pd.DataFrame(data={
                            "name"  : ["Bob", "Carol", "Eve", "Frank"],
                            "phone" : ["919 555-1111", "919 555-2222", "919 555-3333", "919 555-4444"]
    }
)
phone

,name,phone
0,Bob,919 555-1111
1,Carol,919 555-2222
2,Eve,919 555-3333
3,Frank,919 555-4444


Adding the tables to the database `employees.sqlite`:

In [24]:
with sqlite3.connect("employees.sqlite") as conn:
    employees.to_sql("employees", conn, if_exists = "replace", index = False)
    phone.to_sql("phone", conn, if_exists = "replace", index = False)    

We can get the list of tables:

In [25]:
with sqlite3.connect("employees.sqlite") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    print(cursor.fetchall())

[('employees',), ('phone',)]


In [26]:
with sqlite3.connect("employees.sqlite") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
    for c in cursor.fetchall():
        print(c[0])

CREATE TABLE "employees" (
"name" TEXT,
  "email" TEXT,
  "salary" INTEGER,
  "dept" TEXT
)
CREATE TABLE "phone" (
"name" TEXT,
  "phone" TEXT
)


`pd.read_sql_query()` is the function to take the result into a dataframe.

In [27]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM employees", conn)
df

,name,email,salary,dept
0,Alice,alice@company.com,52000,Accounting
1,Bob,bob@company.com,40000,Accounting
2,Carol,carol@company.com,30000,Sales
3,Dave,dave@company.com,33000,Accounting
4,Eve,eve@company.com,44000,Sales
5,Frank,frank@comany.com,37000,Sales


In [28]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM phone", conn)
df

,name,phone
0,Bob,919 555-1111
1,Carol,919 555-2222
2,Eve,919 555-3333
3,Frank,919 555-4444


In [29]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT name AS first_name, salary FROM employees;", conn)
df

,first_name,salary
0,Alice,52000
1,Bob,40000
2,Carol,30000
3,Dave,33000
4,Eve,44000
5,Frank,37000


In [30]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT name AS first_name, salary FROM employees ORDER BY salary;", conn)
df

,first_name,salary
0,Carol,30000
1,Dave,33000
2,Frank,37000
3,Bob,40000
4,Eve,44000
5,Alice,52000


In [31]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT name AS first_name, salary FROM employees ORDER BY salary DESC;", conn)
df

,first_name,salary
0,Alice,52000
1,Eve,44000
2,Bob,40000
3,Frank,37000
4,Dave,33000
5,Carol,30000


In [32]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM employees WHERE salary < 40000;", conn)
df

,name,email,salary,dept
0,Carol,carol@company.com,30000,Sales
1,Dave,dave@company.com,33000,Accounting
2,Frank,frank@comany.com,37000,Sales


In [33]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM employees GROUP BY dept;", conn)
df

,name,email,salary,dept
0,Alice,alice@company.com,52000,Accounting
1,Carol,carol@company.com,30000,Sales


Aggregation by group. There are a number of aggregation functions predefined in SQL. See, for example: https://www.w3schools.com/sql/sql_aggregate_functions.asp


In [34]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT dept, AVG(salary) AS avg_salary FROM employees GROUP BY dept;", conn)
df

,dept,avg_salary
0,Accounting,41666.666667
1,Sales,37000.000000


In [35]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT dept, COUNT(*) AS num_employees FROM employees GROUP BY dept;", conn)
df

,dept,num_employees
0,Accounting,3
1,Sales,3


In [36]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM employees LIMIT 3;", conn)
df

,name,email,salary,dept
0,Alice,alice@company.com,52000,Accounting
1,Bob,bob@company.com,40000,Accounting
2,Carol,carol@company.com,30000,Sales


In [37]:
with sqlite3.connect("employees.sqlite") as conn:
    df = pd.read_sql_query("SELECT * FROM employees ORDER BY name DESC LIMIT 3;", conn)
df

,name,email,salary,dept
0,Frank,frank@comany.com,37000,Sales
1,Eve,eve@company.com,44000,Sales
2,Dave,dave@company.com,33000,Accounting


These are the quiz problems for next Monday. 

**QUIZ 1** Then, what would be the SQL equivalent of the following command?

```python
temperatures["id"][temperatures["year"] == 1990]
```


This isn't very useful when we are just querying a single column, but we can also get multiple columns, in which case the tuple representation makes a bit more sense. 

**QUIZ 2** We can include multiple criteria in `WHERE` using the Boolean operators `AND` and `OR`. How do we take the temperature data from January 1990?

In [38]:
cursor.execute("SQL COMMAND HERE")
result = [cursor.fetchone() for i in range(10)]
result

OperationalError: near "SQL": syntax error

**QUIZ 3** Which stations are either far south or far north? i.e., Latitude greater than 80 degrees or less than -80 degrees?
<!--SELECT id FROM stations WHERE latitude > 80 or latitude< -80 -->

In [ ]:
cursor.execute("SQL COMMAND HERE")
result = [cursor.fetchone() for i in range(10)]  # get just the first 10 results
result